# Timing for td_Qobj

Made by 

In [1]:
import qutip as qt
import numpy as np
from qutip import td_qobj
%load_ext cython

In [2]:
N = 10
destroy, create, Id = qt.destroy(N), qt.create(N), qt.qeye(N)
def exp_i(t,args):
    return np.exp(-1j*t)
def cos_w(t,args):
    return np.cos(args["w"]*t)
tlist = np.linspace(0,10,10000)
tlistlog = np.logspace(-3,1,10000)

# state vector as np array
vec = np.arange(N)*.5+.5j
vec_super = np.arange(N**2)*.5+.5j

# Construct td_Qobj of all type
td_cte1 = td_qobj.td_Qobj(Id)
td_cte2 = td_qobj.td_Qobj([Id])

td_func = td_qobj.td_Qobj([Id,[create,exp_i],[destroy,cos_w]],args={"w":2})
td_str = td_qobj.td_Qobj([Id,[create,"exp(-1j*t)"],[destroy,"cos(w*t)"]],args={'w':2.})
td_array = td_qobj.td_Qobj([Id,[create,np.exp(-1j*tlist)],[destroy,np.cos(2*tlist)]],tlist=tlist)
td_array_log = td_qobj.td_Qobj([Id,[create,np.exp(-1j*tlistlog)],[destroy,np.cos(2*tlistlog)]],tlist=tlistlog)

td_super = qt.liouvillian(td_func, c_ops=td_cte1)

## Compilation and speed

Timing check for all coeff formats.  
Still use the old "get_compiled_call" which kept the python and cython function separated.  
Now once compiled, the cython method is used by the usual method.

In [3]:
# The compiled version build the scipy object from the sparce matrix if not used with data=True
print("Time for cte")
td_cte1.compiled = False
%timeit td_cte1(2)
%timeit td_cte1(2, data=True)
print("Time for compiled cte")
td_cte1.compile()
%timeit td_cte1(2)
%timeit td_cte1(2, data=True)

Time for cte
51.5 µs ± 445 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
11.8 µs ± 187 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
Time for compiled cte
63.6 µs ± 235 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
10.2 µs ± 22.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
print("Time for str")
td_str.compiled = False
%timeit td_str(2)
%timeit td_str(2, data=True)
td_str.compile()
%timeit td_str(2)
%timeit td_str(2, data=True)

Time for str
518 µs ± 883 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
134 µs ± 584 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
70.3 µs ± 874 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
14.6 µs ± 150 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [5]:
print("Time for function")
td_func.compiled = False
%timeit td_func(2)
%timeit td_func(2, data=True)
td_func.compile()
%timeit td_func(2)
%timeit td_func(2, data=True)

Time for function
529 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
142 µs ± 518 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
78.4 µs ± 351 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
22.7 µs ± 140 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
print("Time for array")
td_array.compiled = False
%timeit td_array(2)
%timeit td_array(2, data=True)
td_array.compile()
%timeit td_array(2)
%timeit td_array(2, data=True)

Time for array
583 µs ± 2.66 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
192 µs ± 380 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
69.3 µs ± 660 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
14.4 µs ± 30.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [7]:
print("Time for array with variable time step")
td_array_log.compiled = False
%timeit td_array_log(2)
%timeit td_array_log(2, data=True)
td_array_log.compile()
%timeit td_array_log(2)
%timeit td_array_log(2, data=True)

Time for array with variable time step
586 µs ± 7.14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
190 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
69.1 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
14.6 µs ± 65.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing for rhs vs spmv

In [8]:
print("timing for rhs: cte")
td_cte1.compiled = False
%timeit qt.cy.spmv(td_cte1(2, data=True), vec)
%timeit qt.cy.spmv(Id.data, vec)
%timeit td_cte1.rhs(2,vec)
td_cte1.compile()
%timeit td_cte1.rhs(2,vec)

timing for rhs: cte
19.1 µs ± 929 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
4.8 µs ± 13.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
20.7 µs ± 173 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
5.8 µs ± 16.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
print("timing for rhs: str")
td_str.compiled = False
%timeit qt.cy.spmv(td_str(2, data=True), vec)
%timeit td_str.rhs(2,vec)
td_str.compile()
%timeit td_str.rhs(2,vec)

timing for rhs: str
145 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
147 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
8.76 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [10]:
print("timing for rhs: func")
td_func.compiled = False
%timeit qt.cy.spmv(td_func(2, data=True), vec)
%timeit td_func.rhs(2,vec)
td_func.compile()
%timeit td_func.rhs(2,vec)

timing for rhs: func
153 µs ± 831 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
155 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
15.5 µs ± 87.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
print("timing for rhs: array")
td_array.compiled = False
%timeit qt.cy.spmv(td_array(2, data=True), vec)
%timeit td_array.rhs(2,vec)
td_array.compile()
%timeit td_array.rhs(2,vec)

timing for rhs: array
206 µs ± 6.61 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
202 µs ± 4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
8.39 µs ± 31.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
print("timing for rhs: array with variable time step")
td_array_log.compiled = False
%timeit qt.cy.spmv(td_array_log(2, data=True), vec)
%timeit td_array_log.rhs(2,vec)
td_array_log.compile()
%timeit td_array_log.rhs(2,vec)

timing for rhs: array with variable time step
202 µs ± 5.72 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
202 µs ± 2.66 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
8.6 µs ± 169 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing for expect

In [13]:
print("timing for expect: cte")
td_cte1.compiled = False
%timeit qt.cy.cy_expect_psi(td_cte1(2, data=True), vec, 0)
%timeit td_cte1.expect(2,vec,0)
td_cte1.compile()
%timeit td_cte1.expect(2,vec,0)

timing for expect: cte
16.2 µs ± 39.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
19.1 µs ± 45.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
6.51 µs ± 25.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [14]:
print("timing for expect: str")
td_str.compiled = False
%timeit qt.cy.cy_expect_psi(td_str(2, data=True), vec, 0)
%timeit td_str.expect(2,vec,0)
td_str.compile()
%timeit td_str.expect(2,vec,0)

timing for expect: str
139 µs ± 304 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
142 µs ± 190 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
9.66 µs ± 208 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [15]:
print("timing for expect: func")
td_func.compiled = False
%timeit qt.cy.cy_expect_psi(td_func(2, data=True), vec, 0)
%timeit td_func.expect(2,vec,0)
td_func.compile()
%timeit td_func.expect(2,vec,0)

timing for expect: func
147 µs ± 274 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
152 µs ± 2.61 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
16.9 µs ± 38.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [16]:
print("timing for expect: array")
td_array.compiled = False
%timeit qt.cy.cy_expect_psi(td_array(2, data=True), vec, 0)
%timeit td_array.expect(2,vec,0)
td_array.compile()
%timeit td_array.expect(2,vec,0)

timing for expect: array
201 µs ± 3.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
207 µs ± 17 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
9.41 µs ± 167 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [17]:
print("timing for expect: array logscale")
td_array_log.compiled = False
%timeit qt.cy.cy_expect_psi(td_array_log(2, data=True), vec, 0)
%timeit td_array_log.expect(2,vec,0)
td_array_log.compile()
%timeit td_array_log.expect(2,vec,0)

timing for expect: array logscale
198 µs ± 1.48 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
200 µs ± 366 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
9.43 µs ± 53 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [18]:
print("timing for expect_rho: func")
td_super.compiled = False
%timeit qt.cy.cy_expect_rho_vec(td_super(2, data=True), vec_super, 0)
%timeit td_super.expect(2,vec_super,0)
td_super.compile()
%timeit td_super.expect(2,vec_super,0)

timing for expect_rho: func
155 µs ± 289 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
159 µs ± 918 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
15 µs ± 282 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing with_args

In [19]:
td_args = td_qobj.td_Qobj([Id,[destroy,cos_w]],args={"w":1})

In [20]:
%timeit td_args.with_args(2,{"w":2})
%timeit td_args(2)
%timeit td_args.with_args(2,{"w":2},data=True)
%timeit td_args(2,data=True)
td_args.compile()
%timeit td_args.with_args(2,{"w":2})
%timeit td_args(2)
%timeit td_args.with_args(2,{"w":2},data=True)
%timeit td_args(2,data=True)

284 µs ± 1.31 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
280 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
82.4 µs ± 1.41 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
76.6 µs ± 261 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
76 µs ± 342 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
72 µs ± 137 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
21.6 µs ± 55.4 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
19 µs ± 258 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [21]:
td_args = td_qobj.td_Qobj([Id,[destroy,"cos(w*t)"]],args={"w":1})

In [22]:
%timeit td_args.with_args(2,{"w":2})
%timeit td_args(2)
%timeit td_args.with_args(2,{"w":2},data=True)
%timeit td_args(2,data=True)
td_args.compile()
%timeit td_args.with_args(2,{"w":2})
%timeit td_args(2)
%timeit td_args.with_args(2,{"w":2},data=True)
%timeit td_args(2,data=True)

290 µs ± 2.36 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
285 µs ± 3.32 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
77.5 µs ± 1.53 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
72.4 µs ± 277 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
72.3 µs ± 221 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
68.3 µs ± 1.65 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
18.1 µs ± 214 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
13.7 µs ± 95.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Timing with_state

In [23]:
def coeff_state(t, psi, args):
    return np.max(psi)*args["w"]
td_state = td_qobj.td_Qobj([Id, [destroy, coeff_state]],args={"w":1})
%timeit td_state.with_state(2,vec)
td_state.compile()
%timeit td_state.with_state(2,vec)

300 µs ± 9.43 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
78.1 µs ± 1.85 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
